# Teste Unitário - Datamart

**Objetivo**: Gerar as bases amostrais do pacote de datamart

---

## Cabeçalho 

### Imports 

In [2]:
import os
import pyunpack
import rarfile
import shutil
import zipfile
import numpy as np
import pandas as pd

from pathlib import Path
from tqdm import tqdm

### Configurações 

In [3]:
%config Completer.use_jedi = False
%load_ext autoreload
%autoreload 2

### Caminhos 

In [4]:
try:
    PASTA_NOTEBOOK / ""
except:
    PASTA_NOTEBOOK = Path(os.getcwd())
    
PASTA_PROJETO = PASTA_NOTEBOOK.parent
PASTA_DADOS = PASTA_PROJETO / "dados"
PASTA_COMPLETO = PASTA_DADOS / "completo"
PASTA_EXTERNO = PASTA_COMPLETO / "externo"
PASTA_TESTE = PASTA_PROJETO / "src/tests"
PASTA_TESTE_DADOS = PASTA_TESTE / "dados"
PASTA_TESTE_SAIDA = PASTA_TESTE / "saida"
PASTA_TESTE_CODIGO = PASTA_TESTE / "codigos"
PASTA_TESTE_AQUIS = PASTA_TESTE / "aquisicao"
PASTA_TESTE_DM = PASTA_TESTE / "datamart"

os.chdir(PASTA_PROJETO)

### Código Próprio 

In [8]:
os.chdir(PASTA_PROJETO)
import src.io.le_dados as le_dados
import src.io.caminho as caminho
import src.io.data_store as data_store
import src.aquisicao as aquis
import src.aquisicao.executa as aquis_ex

---

## DataStore 

In [6]:
ds = data_store.DataStore("teste")

---

## Aquisição 

### INEP 

In [ ]:
for ano in range(2007, 2021):
    for etl in ["escola", "docente", "matricula", "gestor", "turma"]:
        if etl == "gestor" and ano <= 2018:
            continue
        aquis_ex.executa_etl_microdado_inep(etl, ds, str(ano), True, True)
aquis_ex.executa_etl("ideb", ds, True, True)

---

## IDEB 

In [ ]:
caminho_entrada = PASTA_EXTERNO / "ideb"
caminho_saida = PASTA_TESTE_DADOS / "externo/ideb"
caminho_saida.mkdir(exist_ok=True, parents=True)

for f in tqdm(os.listdir(caminho_entrada)):
    nome = os.path.splitext(f)[0]
    with zipfile.ZipFile(caminho_entrada / f) as z:
        data = pd.read_excel(z.open(f"{nome}/{nome}.xlsx"), header=None)
        sample = (
            data.head(10)
            .append(data[data[3].astype("str").isin(escolas.astype("str"))])
            .append(data.tail(3))
        )
        
    with zipfile.ZipFile(
        caminho_saida / f, 
        mode="w", 
        compression=zipfile.ZIP_DEFLATED, 
        compresslevel=9
    ) as zout:
        sample.to_excel(caminho_saida / f"{nome}.xlsx", header=None, index=False)
        zout.write(caminho_saida / f"{nome}.xlsx", f"{nome}/{nome}.xlsx")
        os.remove(caminho_saida / f"{nome}.xlsx")

---